In [106]:
from gensim import corpora, models, similarities
import pickle
import string

In [107]:
def remove_punctuation(x):
    x = str(x)
    return x.translate(string.maketrans('',''),string.punctuation)

In [108]:
beer_reviews = pickle.load(open('beer_reviews.pkl','rb'))

In [109]:
documents = [reviews[1] for reviews in beer_reviews.items()]

In [110]:
documents = [' '.join(review) for review in documents]

In [111]:
documents = [remove_punctuation(doc) for doc in documents]

In [113]:
documents = [review.lower() for review in documents]

In [115]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]for document in documents]

In [116]:
from collections import defaultdict

In [117]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]for text in texts]

In [118]:
dictionary = corpora.Dictionary(texts)

In [119]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [120]:
tfidf = models.TfidfModel(corpus)

In [121]:
corpus_tfidf = tfidf[corpus]

In [122]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=500)

In [123]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [86]:
beer_names = beer_reviews.keys()

In [94]:
doc = ''
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]

In [124]:
doc = beer_reviews['Lagunitas IPA']
vec_bow = dictionary.doc2bow([word.lower() for word in doc])
vec_lsi = lsi[vec_bow]

In [125]:
sims = index[vec_lsi]
for beer in sorted(enumerate(sims), key = lambda x: -x[1])[:5]:
    print(beer[0])

0
1
2
3
4
